In [ ]:
import os
import string
import requests
import glob
import pandas as pd 
from bs4 import BeautifulSoup
from PIL import Image

In [ ]:
path = "lexile/raw/book-reviews/"
files = [f for f in glob.glob(path + "*.html", recursive=True)]

In [ ]:
file_names = []

for file in files:
    f = file.split("/")[3]
    file_names.append(f[:-5])

In [ ]:
csm_ages = []
cover_src = []

for file in files:
    with open(file, 'r') as f: 
        soup = BeautifulSoup(f.read(), 'html.parser')
        csm_ages.append(soup.find(class_="csm-green-age").get_text().replace("age ", "")[:-1])
        cover = soup.find(class_="field field-name-field-product-image field-type-image field-label-hidden")
        cover_src.append(cover.find("img").get('src'))

In [ ]:
filler = ["_" for _ in range(len(csm_ages))]

In [ ]:
names = [x+y+z for x,y,z in zip(csm_ages,filler,file_names)]

In [ ]:
info = dict(zip(names, cover_src))

In [ ]:
!mkdir 'covers'

In [ ]:
for file in files:
    with open(file, 'r') as f: 
        soup = BeautifulSoup(f.read(), 'html.parser')
        for k, v in info.items():
            if ".jpeg?" in v:
                if not os.path.isfile("./covers/" + k + '.jpeg'):
                    with open('./covers/' + k + '.jpeg', 'wb') as f:
                            f.write(requests.get(v).content)
            elif '.jpg?' in v:
                if not os.path.isfile("./covers/" + k + '.jpg'):
                    with open('./covers/' + k + '.jpg', 'wb') as f:
                        f.write(requests.get(v).content)
            elif '.png?' in v:
                if not os.path.isfile("./covers/" + k + '.png'):
                    with open('./covers/' + k + '.png', 'wb') as f:
                            f.write(requests.get(v).content)   

In [ ]:
!mkdir covers-jpg

In [ ]:
path2 = "covers/*"
path3 = "covers-jpg/"
files = [f for f in glob.glob(path2, recursive=True)]

In [ ]:
def convert (file):
    im = Image.open(file)
    rgb_im = im.convert('RGB')
    new_file = rgb_im.save(path3+file[7:].split(".")[0]+".jpg")
    return new_file

In [ ]:
error = []

for file in files:
    try:
        convert(file)
    except OSError:
        error.append(file)

In [ ]:
error

In [ ]:
for e in error:
    os.remove(e)